# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [637]:
problem = '''The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits.'''

## 2. Ask for parameters

In [647]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [648]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

**Production Quantities:**

*  **x1:**  Units of product P1 produced and sold weekly.
*  **x2:** Units of product P2 produced and sold weekly.
*  **x3:** Units of product P3 produced and sold weekly.

**Subcontracting Quantities:**

*  **s1:** Units of product P1 subcontracted for smelting weekly.
*  **s2:** Units of product P2 subcontracted for smelting weekly. 


# 2. Ask for objective

In [649]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [650]:
Markdown(response2.text)

**Objective Function (Maximize Weekly Profits):**

```
Maximize: 
  (1.50 - 0.30 - 0.20 - 0.30) * (x1 - s1) +  // Profit from P1 produced in-house
  (1.50 - 0.50 - 0.20 - 0.30) * s1 +          // Profit from P1 subcontracted
  (1.80 - 0.50 - 0.10 - 0.20) * (x2 - s2) +  // Profit from P2 produced in-house
  (1.80 - 0.60 - 0.10 - 0.20) * s2 +          // Profit from P2 subcontracted 
  (1.97 - 0.40 - 0.27 - 0.20) * x3             // Profit from P3 (no subcontracting) 
```

**Explanation:**

The objective function calculates the total profit by summing the profit contributions from each product. 

* **Profit per unit:** Calculated as (selling price - cost of smelting - cost of mechanization - cost of assembly & packaging).
* **Subcontracting:**  The function considers both in-house production (x1 - s1) and subcontracted production (s1) for P1 and P2 to account for the different cost structures.

This objective function will be maximized subject to various constraints (not asked for in this question) related to production capacity, subcontracting limits, and non-negative production quantities. 


# 3. Ask for constraints

In [651]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [652]:
Markdown(response3.text)

## Constraints:

**1. Production Capacity Constraints:**

* **Smelting Capacity:**
   *  6(x1 - s1) + 10(x2 - s2) + 8x3 ≤ 8000  (minutes) 
* **Mechanisation Capacity:**
   * 6x1 + 3x2 + 8x3 ≤ 12000 (minutes)
* **Assembly and Packaging Capacity:**
   * 3x1 + 2x2 + 2x3 ≤ 10000 (minutes)

**2. Subcontracting Constraints:**

* **Subcontracted Smelting Cannot Exceed Production:**
   * s1 ≤ x1 
   * s2 ≤ x2

**3. Non-Negativity Constraints:**

* x1, x2, x3, s1, s2 ≥ 0 


## 3. Generate the pyomo code

In [653]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [654]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Create a Concrete Model
model = pyo.ConcreteModel()

# Define Variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P1 produced and sold
model.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P2 produced and sold
model.x3 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P3 produced and sold
model.s1 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P1 subcontracted for smelting
model.s2 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P2 subcontracted for smelting

# Objective Function
model.profit = pyo.Objective(
    expr= (1.50 - 0.30 - 0.20 - 0.30) * (model.x1 - model.s1) + 
          (1.50 - 0.50 - 0.20 - 0.30) * model.s1 + 
          (1.80 - 0.50 - 0.10 - 0.20) * (model.x2 - model.s2) + 
          (1.80 - 0.60 - 0.10 - 0.20) * model.s2 + 
          (1.97 - 0.40 - 0.27 - 0.20) * model.x3, 
    sense=pyo.maximize
)

# Constraints
model.smelting_capacity = pyo.Constraint(
    expr= 6*(model.x1 - model.s1) + 10*(model.x2 - model.s2) + 8*model.x3 <= 8000
)

model.mechanization_capacity = pyo.Constraint(
    expr= 6*model.x1 + 3*model.x2 + 8*model.x3 <= 12000
)

model.assembly_capacity = pyo.Constraint(
    expr= 3*model.x1 + 2*model.x2 + 2*model.x3 <= 10000
)

model.subcontracting_p1 = pyo.Constraint(
    expr= model.s1 <= model.x1
)

model.subcontracting_p2 = pyo.Constraint(
    expr= model.s2 <= model.x2
)

# Solve the Model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if needed
solver.solve(model)

# Print the Results
print("Optimal Solution:")
print(f"Produce and sell {model.x1()} units of P1")
print(f"Produce and sell {model.x2()} units of P2")
print(f"Produce and sell {model.x3()} units of P3")
print(f"Subcontract {model.s1()} units of P1 for smelting")
print(f"Subcontract {model.s2()} units of P2 for smelting")
print(f"Maximum Weekly Profit: ${model.profit()}") 
``` 


## 4. Input problem data and try running the generated code

In [655]:
import pyomo.environ as pyo

# Create a Concrete Model
model = pyo.ConcreteModel()

# Define Variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P1 produced and sold
model.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P2 produced and sold
model.x3 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P3 produced and sold
model.s1 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P1 subcontracted for smelting
model.s2 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P2 subcontracted for smelting

# Objective Function
model.profit = pyo.Objective(
    expr= (1.50 - 0.30 - 0.20 - 0.30) * (model.x1 - model.s1) + 
          (1.50 - 0.50 - 0.20 - 0.30) * model.s1 + 
          (1.80 - 0.50 - 0.10 - 0.20) * (model.x2 - model.s2) + 
          (1.80 - 0.60 - 0.10 - 0.20) * model.s2 + 
          (1.97 - 0.40 - 0.27 - 0.20) * model.x3, 
    sense=pyo.maximize
)

# Constraints
model.smelting_capacity = pyo.Constraint(
    expr= 6*(model.x1 - model.s1) + 10*(model.x2 - model.s2) + 8*model.x3 <= 8000
)

model.mechanization_capacity = pyo.Constraint(
    expr= 6*model.x1 + 3*model.x2 + 8*model.x3 <= 12000
)

model.assembly_capacity = pyo.Constraint(
    expr= 3*model.x1 + 2*model.x2 + 2*model.x3 <= 10000
)

model.subcontracting_p1 = pyo.Constraint(
    expr= model.s1 <= model.x1
)

model.subcontracting_p2 = pyo.Constraint(
    expr= model.s2 <= model.x2
)

# Solve the Model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if needed
solver.solve(model)

# Print the Results
print("Optimal Solution:")
print(f"Produce and sell {model.x1()} units of P1")
print(f"Produce and sell {model.x2()} units of P2")
print(f"Produce and sell {model.x3()} units of P3")
print(f"Subcontract {model.s1()} units of P1 for smelting")
print(f"Subcontract {model.s2()} units of P2 for smelting")
print(f"Maximum Weekly Profit: ${model.profit()}") 

Optimal Solution:
Produce and sell 0.0 units of P1
Produce and sell 4000.0 units of P2
Produce and sell 0.0 units of P3
Subcontract 0.0 units of P1 for smelting
Subcontract 3200.0 units of P2 for smelting
Maximum Weekly Profit: $3680.0000000000005


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [656]:
print(response.text)

## Variables:

**Production Quantities:**

*  **x1:**  Units of product P1 produced and sold weekly.
*  **x2:** Units of product P2 produced and sold weekly.
*  **x3:** Units of product P3 produced and sold weekly.

**Subcontracting Quantities:**

*  **s1:** Units of product P1 subcontracted for smelting weekly.
*  **s2:** Units of product P2 subcontracted for smelting weekly. 



In [657]:
print(response2.text)

**Objective Function (Maximize Weekly Profits):**

```
Maximize: 
  (1.50 - 0.30 - 0.20 - 0.30) * (x1 - s1) +  // Profit from P1 produced in-house
  (1.50 - 0.50 - 0.20 - 0.30) * s1 +          // Profit from P1 subcontracted
  (1.80 - 0.50 - 0.10 - 0.20) * (x2 - s2) +  // Profit from P2 produced in-house
  (1.80 - 0.60 - 0.10 - 0.20) * s2 +          // Profit from P2 subcontracted 
  (1.97 - 0.40 - 0.27 - 0.20) * x3             // Profit from P3 (no subcontracting) 
```

**Explanation:**

The objective function calculates the total profit by summing the profit contributions from each product. 

* **Profit per unit:** Calculated as (selling price - cost of smelting - cost of mechanization - cost of assembly & packaging).
* **Subcontracting:**  The function considers both in-house production (x1 - s1) and subcontracted production (s1) for P1 and P2 to account for the different cost structures.

This objective function will be maximized subject to various constraints (not asked for in thi

In [658]:
print(response3.text)

## Constraints:

**1. Production Capacity Constraints:**

* **Smelting Capacity:**
   *  6(x1 - s1) + 10(x2 - s2) + 8x3 ≤ 8000  (minutes) 
* **Mechanisation Capacity:**
   * 6x1 + 3x2 + 8x3 ≤ 12000 (minutes)
* **Assembly and Packaging Capacity:**
   * 3x1 + 2x2 + 2x3 ≤ 10000 (minutes)

**2. Subcontracting Constraints:**

* **Subcontracted Smelting Cannot Exceed Production:**
   * s1 ≤ x1 
   * s2 ≤ x2

**3. Non-Negativity Constraints:**

* x1, x2, x3, s1, s2 ≥ 0 



In [659]:
print(response4.text)

```python
import pyomo.environ as pyo

# Create a Concrete Model
model = pyo.ConcreteModel()

# Define Variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P1 produced and sold
model.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P2 produced and sold
model.x3 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P3 produced and sold
model.s1 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P1 subcontracted for smelting
model.s2 = pyo.Var(within=pyo.NonNegativeReals)  # Units of P2 subcontracted for smelting

# Objective Function
model.profit = pyo.Objective(
    expr= (1.50 - 0.30 - 0.20 - 0.30) * (model.x1 - model.s1) + 
          (1.50 - 0.50 - 0.20 - 0.30) * model.s1 + 
          (1.80 - 0.50 - 0.10 - 0.20) * (model.x2 - model.s2) + 
          (1.80 - 0.60 - 0.10 - 0.20) * model.s2 + 
          (1.97 - 0.40 - 0.27 - 0.20) * model.x3, 
    sense=pyo.maximize
)

# Constraints
model.smelting_capacity = pyo.Constraint(
    expr= 6*(model.x1 - model.s1) + 10*(model